<a href="https://colab.research.google.com/github/ganesh-swaminathan/pythonds/blob/main/uber_casestudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import numpy as np
import pandas as pd
from math import sin, cos, sqrt, atan2
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error , r2_score


In [44]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/Othercomputers/My Mac/epgdsai/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/My Mac/epgdsai/data


Reading the CSV file

In [45]:
uberdata = pd.read_csv('uber.csv')

In [46]:
uberdata

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...,...,...
199995,42598914,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1
199996,16382965,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1
199997,27804658,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2
199998,20259894,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1


In [47]:
uberdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         200000 non-null  int64  
 1   key                200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


In [48]:
uberdata.drop(columns=['key', 'Unnamed: 0'], inplace=True)

In [49]:
uberdata['pickup_datetime'] = pd.to_datetime(uberdata['pickup_datetime'])

In [50]:
uberdata['year'] = uberdata['pickup_datetime'].dt.year

In [51]:
uberdata['month'] = uberdata['pickup_datetime'].dt.month
uberdata['weekday'] = uberdata['pickup_datetime'].dt.weekday
uberdata['hour'] = uberdata['pickup_datetime'].dt.hour

In [52]:
uberdata.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,2015,5,3,19
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2009,7,4,20
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,2009,8,0,21
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,2009,6,4,8
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,2014,8,3,17


In [53]:
uberdata.isna().sum()

fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
year                 0
month                0
weekday              0
hour                 0
dtype: int64

In [54]:
uberdata.dropna(inplace=True)

In [55]:
uberdata['year'].unique()

array([2015, 2009, 2014, 2011, 2012, 2010, 2013])

In [56]:
uberdata['month'].value_counts()

5     18859
3     18763
4     18606
6     17787
1     17668
2     16695
10    16212
12    15516
11    15312
9     15266
7     15094
8     14221
Name: month, dtype: int64

In [57]:
uberdata['Period']=pd.cut(uberdata['month'],bins=[0,3,6,9,12],
                               labels=['Q1','Q2','Q3','Q4'])

In [58]:
uberdata.sample(15)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,Period
52130,6.1,2009-06-24 22:23:41+00:00,-73.970383,40.785667,-73.959943,40.771155,3,2009,6,2,22,Q2
190020,3.7,2011-06-14 07:51:00+00:00,-73.979413,40.781747,-73.973545,40.785372,1,2011,6,1,7,Q2
21574,5.7,2010-03-19 22:00:52+00:00,-73.954947,40.773363,-73.968301,40.757336,1,2010,3,4,22,Q1
166830,11.0,2015-05-17 00:56:18+00:00,-73.994789,40.731453,-73.976173,40.751656,5,2015,5,6,0,Q2
4060,2.5,2009-04-28 09:43:31+00:00,-73.973106,40.790165,-73.971922,40.792156,1,2009,4,1,9,Q2
41930,8.5,2012-01-08 09:49:17+00:00,-73.977841,40.764484,-73.965554,40.790454,1,2012,1,6,9,Q1
186508,24.1,2011-12-21 04:29:00+00:00,-74.006288,40.739757,-73.886970,40.763055,4,2011,12,2,4,Q4
144752,18.0,2013-04-05 20:26:05+00:00,-73.968926,40.760958,-73.989858,40.719872,1,2013,4,4,20,Q2
18462,9.0,2015-06-21 10:24:54+00:00,-73.954193,40.764252,-73.983124,40.764175,1,2015,6,6,10,Q2
9921,5.7,2011-09-25 13:57:55+00:00,-73.970197,40.751629,-73.980990,40.733883,1,2011,9,6,13,Q3


In [59]:
uberdata['Hourly_Segments']=pd.cut(uberdata['hour'],bins=[0,3,7,11,15,19,23],
                               labels=['H1','H2','H3','H4','H5','H6'],
                               include_lowest=True)

In [61]:
uberdata.head(15)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,Period,Hourly_Segments
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,2015,5,3,19,Q2,H5
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2009,7,4,20,Q3,H6
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,2009,8,0,21,Q3,H6
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,2009,6,4,8,Q2,H3
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,2014,8,3,17,Q3,H5
5,4.9,2011-02-12 02:27:09+00:00,-73.969019,40.755910,-73.969019,40.755910,1,2011,2,5,2,Q1,H1
6,24.5,2014-10-12 07:04:00+00:00,-73.961447,40.693965,-73.871195,40.774297,5,2014,10,6,7,Q4,H2
7,2.5,2012-12-11 13:52:00+00:00,0.000000,0.000000,0.000000,0.000000,1,2012,12,1,13,Q4,H4
8,9.7,2012-02-17 09:32:00+00:00,-73.975187,40.745767,-74.002720,40.743537,1,2012,2,4,9,Q1,H3
9,12.5,2012-03-29 19:06:00+00:00,-74.001065,40.741787,-73.963040,40.775012,1,2012,3,3,19,Q1,H5


In [62]:
uberdata.shape

(199999, 13)

In [63]:
uberdata = uberdata.query('(pickup_latitude < 90) & (dropoff_latitude < 90) & (pickup_latitude > -90) & (dropoff_latitude > -90) & (pickup_longitude < 180) & (dropoff_longitude < 180) & (pickup_longitude > -180) & (dropoff_longitude > -180)')

In [64]:
uberdata.shape

(199987, 13)

In [66]:
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Difference in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance


In [67]:
uberdata['distance'] = uberdata.apply(lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

<ipython-input-67-26e623699d89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uberdata['distance'] = uberdata.apply(lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)


In [68]:
uberdata.sample(10)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,Period,Hourly_Segments,distance
142665,10.5,2015-06-08 08:01:34+00:00,-73.992912,40.730740,-73.975838,40.755268,2,2015,6,0,8,Q2,H3,3.083529
187034,8.9,2011-06-13 01:25:00+00:00,-73.997173,40.722162,-73.992898,40.752827,1,2011,6,0,1,Q2,H1,3.428763
67321,6.1,2009-03-04 21:04:00+00:00,-73.972840,40.748463,-73.954517,40.769758,1,2009,3,2,21,Q1,H6,2.826414
151400,3.3,2010-06-17 08:14:00+00:00,-73.975523,40.792180,-73.979008,40.787593,1,2010,6,3,8,Q2,H3,0.588414
30427,12.1,2011-06-25 19:35:00+00:00,-73.962988,40.758252,-73.984065,40.785627,1,2011,6,5,19,Q2,H5,3.523623
20266,4.0,2014-01-08 06:17:38+00:00,-73.990498,40.756277,-73.987938,40.758778,1,2014,1,2,6,Q1,H2,0.351898
49686,5.7,2012-05-10 10:44:13+00:00,-73.982058,40.732212,-73.974805,40.748267,1,2012,5,3,10,Q2,H3,1.886918
170748,9.5,2015-03-08 12:43:48+00:00,-74.003113,40.744190,-74.013344,40.717453,1,2015,3,6,12,Q1,H4,3.095508
117064,6.1,2011-12-31 18:54:00+00:00,-74.000765,40.731847,-73.984342,40.724027,1,2011,12,5,18,Q4,H5,1.634398
11107,26.1,2011-09-30 05:19:00+00:00,-73.834117,40.692278,-73.726560,40.723473,1,2011,9,4,5,Q3,H2,9.706979


In [69]:
uberdata.columns

Index(['fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'year',
       'month', 'weekday', 'hour', 'Period', 'Hourly_Segments', 'distance'],
      dtype='object')

In [70]:
uberdata.drop(columns = ['pickup_datetime', 'month', 'hour'], axis=1, inplace=True)

<ipython-input-70-7848656e805d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uberdata.drop(columns = ['pickup_datetime', 'month', 'hour'], axis=1, inplace=True)


In [82]:
uberdata[uberdata.columns[8]]

0         Q2
1         Q3
2         Q3
3         Q2
4         Q3
          ..
199995    Q4
199996    Q1
199997    Q2
199998    Q2
199999    Q2
Name: Period, Length: 199987, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [87]:
le = LabelEncoder()
uberdata['Hourly_Segments'] = le.fit_transform(uberdata['Hourly_Segments'])
uberdata['Period'] = le.fit_transform(uberdata['Period'])

<ipython-input-87-b1c7a97d885c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uberdata['Hourly_Segments'] = le.fit_transform(uberdata['Hourly_Segments'])
<ipython-input-87-b1c7a97d885c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uberdata['Period'] = le.fit_transform(uberdata['Period'])


In [88]:
uberdata.sample(5)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,Period,Hourly_Segments,distance
198641,16.9,-73.994495,40.750550,-73.961692,40.711000,5,2010,3,2,4,5.194246
139811,39.5,-73.954291,40.781237,-73.955914,40.781895,1,2014,4,3,2,0.155007
165265,5.5,-73.980358,40.754242,-73.989755,40.747062,6,2014,2,1,5,1.124275
185929,5.7,-73.948654,40.786534,-73.969889,40.785610,1,2010,3,0,5,1.790760
9017,4.5,-73.986495,40.739899,-73.977407,40.746447,1,2013,5,0,4,1.056561


In [102]:
uberdata = uberdata[uberdata['distance'] != 0.0000]

In [104]:
uberdata['fare_amount'].value_counts()

6.50     9417
4.50     8004
8.50     7328
5.70     5645
5.30     5645
         ... 
31.39       1
33.45       1
73.83       1
44.25       1
89.10       1
Name: fare_amount, Length: 1215, dtype: int64

In [105]:

X = uberdata[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','distance']]
Y = uberdata['fare_amount']

xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=.20)

In [106]:
base_estimator=RandomForestRegressor()

bagging=BaggingRegressor(
    estimator=base_estimator,
    n_estimators=10,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=True,
    verbose=0,
)


In [ ]:
bagging.fit(xtrain,ytrain)

In [ ]:
y_pred = bagging.predict(xtest)

In [ ]:
mean_squared_error(y_pred,ytest)